# SQL

# What is SQL?

SQL (Structured Query Language) is a language for interacting with databases.  Databases store information in an organized way, and the queries to access the data are often written in SQL.  This is especially true if the database uses a Relational Database Management System (RDBMS), which treats the data as being organized in tables.

Even for big data, where before it was considered too expensive to do a full RDBMS system with SQL queries, things have gradually moved in the direction of the greater organization necessary to make these queries.  So even when dealing with big data, the interface will often be similar to SQL.

SQL has multiple implementations, but the one we'll be demonstrating is SQLite.  This python module gives us the power to make SQL queries against an existing database and put the result in a Pandas dataframe.  (These examples use the books.db file and corresponding examples from the textbook.)

An interesting aspect of SQL is that a query specifies what it wants, but not how to get it.  SQLite compiles the query into a program, and chooses among multiple approaches to get the data (for example, what to filter by first when handling an AND).  As a result, what the engine is doing is often treated like a black box by the user.

## Basic setup and column queries

In [ ]:
# For Google colab only
from google.colab import files

uploaded = files.upload() # Get "books.db" file

In [ ]:
import sqlite3
connection = sqlite3.connect('books.db')

import pandas as pd
pd.read_sql('SELECT * FROM authors', connection)

Here, we use SQLite to interact with a file, treating it as the database.  .connect() creates a Connection object, which can then be used as an argument indicating where our database is to be found.  And finally, the string with all the capital letters is SQL, indicating that we want all (*) lines from the authors table.  The result of the read_sql is a DataFrame containing the results of the query.



Note that some SQL systems require a semicolon at the end of each statement, particularly if they accept multiple statements at a time.  But this implementation doesn't.  There are a variety of little differences between SQL implementations that vary from vendor to vendor.

Also note that the caps-lock is a stylistic choice rather than absolutely essential - the queries in SQLite would work as well with different case choices.  But capitalizing keywords makes it easier to see what's happening.

This particular "authors" table is kind of small.  The same database has a "titles" table with more columns.

In [ ]:
pd.read_sql('SELECT * from titles', connection)

To select data from just some of the columns, we can list the desired columns where we put a * before.

In [ ]:
pd.read_sql('SELECT title, edition from titles', connection)

If you're trying to just see the different values for a particular column,
SELECT DISTINCT avoids repeating itself.

In [ ]:
pd.read_sql('SELECT DISTINCT copyright FROM titles', connection)

## Where, Like, and Order By

The WHERE clause allows filtering of values. We'll also move to triple-quote syntax as the queries get long.


In [ ]:
pd.read_sql("""SELECT title, edition from titles WHERE edition > 5""", connection)

Other numerical operators include <, <=, =, >=, =, and != (for not equal).  The conditions can be joined with AND, OR, and NOT.  (Also note that as queries get long,
SQL doesn't mind additional whitespace, including newlines.)

In [ ]:
pd.read_sql("""SELECT title, edition
from titles WHERE edition != 6 AND edition >= 3
AND NOT edition >= 10""", connection)

Note that sometimes columns with numbers are mistyped in the database as being strings, in which case, comparison operators won't work as expected. To fix this, cast the column as an integer before making the comparison, like this:

In [ ]:
pd.read_sql("""SELECT title, edition from titles WHERE cast(edition as int) > 5""", connection)

There isn't any missing data in these tables, but you could also avoid missing entries with WHERE [columnname] IS NOT NULL.  (Null is the value for missing data in SQL.)

WHERE can also look for regular-expression-like patterns using LIKE.  Here we retrieve all the Deitels' books about how to program.  The % acts like a * in other regular expressions, matching zero or more characters.

In [ ]:
pd.read_sql("""SELECT title, edition from titles
            WHERE title LIKE '%How to Program'""", connection)

WHERE also understands NOT LIKE, which gets all the other entries.

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
            WHERE title NOT LIKE '%How to Program'""", connection)

When there are many results, it may matter how they're ordered.  ORDER BY [column] [ASC/DESC] will choose the column and whether it's ascending or descending order.  The default is ascending.

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
               ORDER BY edition DESC""", connection)

In [ ]:
pd.read_sql("""SELECT title, edition from titles
               ORDER BY edition""", connection) # No DESC

LIMIT [number] finds just the top [number] entries in the table, which is useful when dealing with a large table.  It's particularly useful in conjunction with ORDER BY.

In [ ]:
pd.read_sql("""SELECT title, edition FROM titles
              ORDER BY edition DESC LIMIT 3""", connection)

## Quick checks

How would you create a DataFrame that had just the books that start with "Android" from the titles table?

What about selecting titles published in or after 2017 that are at least 3rd edition?

And the 4 smallest edition numbers?

## Joins

Now we'll take a look at joins, starting with the small author_ISBN table.  (ISBNs are codes that uniquely identify books.)

In [ ]:
pd.read_sql("SELECT * FROM author_ISBN", connection)

The provided author_ISBN table is somewhat cryptic as-is, but its id column refers to an author from the author table.  (This makes the id in the author_ISBN table a "foreign key."  In the table where it uniquely identifies authors, the id is the "primary key.")  The query can "join" multiple tables and use this key to match the data.

In [ ]:
pd.read_sql("""SELECT first, last, isbn 
               FROM authors
               INNER JOIN author_ISBN 
                  ON authors.id = author_ISBN.id
               ORDER BY last,first""", connection)

Notice that isbn was not in the authors table, but we could include it because we were performing the INNER JOIN.  There are also a lot more entries here than in the authors table, which only listed each author once.  The database's use of tables helps it avoid redundancy in how it's using memory.

We can join across multiple tables in this way, but if columns appear in multiple tables, like isbn here, we have to be careful to specify which table the columns belong to, as in the SELECT below.  Otherwise, we'll get an error saying the query was ambiguous.

In [ ]:
pd.read_sql("""SELECT first, last, titles.isbn, title
               FROM authors 
               INNER JOIN author_ISBN 
                  ON authors.id = author_ISBN.id
               INNER JOIN titles
                  ON author_ISBN.isbn = titles.isbn
               ORDER BY last,first""", connection)

Besides INNER JOIN, SQL supports a few other joins that differ in how they treat missing data.  In an INNER JOIN, only records with keys in both tables are joined.  In a FULL JOIN, aka FULL OUTER JOIN, all records are surfaced, but null values are given for missing data when one side is present and the other is not.  A LEFT JOIN or a RIGHT JOIN privileges the first or second table, showing all entries for that table and giving null values when the other table doesn't have a match.  On the whole, INNER JOIN is probably the most useful.

# Inserting, changing, and deleting data

It's also possible to insert data into the table with SQL.  The object that does this is called a cursor, and it can generally be used to interact with the database in specific places.  An INSERT INTO can add a line, while an UPDATE can update the values of a particular row.

In [ ]:
cursor = connection.cursor()
cursor = cursor.execute("""INSERT INTO authors (first, last)
                           VALUES ('John', 'Doe')""")

pd.read_sql("SELECT * from authors", connection)

In [ ]:
cursor = cursor.execute("""UPDATE authors SET first='Jane' WHERE first='John' AND last='Doe'""")
pd.read_sql("SELECT * from authors", connection)

You can also delete entries from the database with DELETE FROM.

In [ ]:
cursor = cursor.execute("DELETE FROM authors WHERE first='Dan'")
pd.read_sql("SELECT * from authors", connection)

Out of all of these commands, SELECT and WHERE are the commands you're most likely to interact with, pulling the data you want and then manipulating the data on your own end within the DataFrame.  These useful basic operations are also the ones you'd be likely to use in a big distributed database.

When you're done with the SQL connection, you should close it.

In [ ]:
connection.close()

# Dremel, BigQuery, Drill

Google uses a SQL interface to its logs, called Dremel.  It made available a public version of Dremel roughly 10 years ago, and Apache's version of it is called Drill.  The queries in these systems use standard SQL with only slight changes, but the search for the answer to the query is distributed across many servers, allowing a very parallel search for the query results.  The speed requires several optimizations in how the data is stored, such as storing the columns together in storage instead of storing complete records together.  But the user only needs to know the SQL commands to successfully use it.

While it seemed at first as if Big Data might not use SQL, big players like Google use it to interact with their data in a structured way.